In [1]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import re
from collections import namedtuple
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [2]:
#Settings
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
#Read the preprocessed Data
filepath = './CICIDS2017_preprocessed'
data = pd.read_csv(filepath)

In [4]:
data.shape

(2830743, 21)

In [5]:
data.head()

,Destination Port,Flow Duration,Fwd Packet Length Max,Bwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Mean,Fwd IAT Min,Bwd IAT Mean,Bwd IAT Std,Fwd Packets/s,Packet Length Mean,Packet Length Std,PSH Flag Count,Average Packet Size,Avg Bwd Segment Size,Subflow Fwd Packets,Init_Win_bytes_forward,Init_Win_bytes_backward,Active Std,Label_binary
0,0.000336,1.491667e-06,0.000000,0.0,0.000000e+00,1.500000e-06,0.000000,9.999999e-08,0.000000e+00,0.0,2.008032e-03,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.004440,0.003723,0.000000,0
1,0.917800,7.999999e-07,0.000000,0.0,1.250775e-07,4.000000e-07,0.000000,9.999999e-08,4.083333e-07,0.0,4.016064e-03,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.003723,0.004440,0.000000,0
2,0.001877,8.329999e-04,0.001934,0.0,0.000000e+00,8.330082e-04,0.000000,9.999999e-08,0.000000e+00,0.0,3.335101e-06,0.014384,0.0,0.0,0.018493,0.008275,0.000000,0.000000,0.000000,0.000000,0
3,0.001877,3.085833e-04,0.001934,0.0,0.000000e+00,3.085916e-04,0.000000,9.999999e-08,0.000000e+00,0.0,9.004872e-06,0.014384,0.0,0.0,0.018493,0.008275,0.000000,0.000000,0.000000,0.000000,0
4,0.000000,9.263445e-01,0.000000,0.0,2.995055e-02,1.166667e-07,0.006345,9.999999e-08,0.000000e+00,0.0,4.408007e-07,0.000000,0.0,0.0,0.000000,0.000000,0.000664,0.000000,0.000000,0.028615,0


In [6]:
Attack_data = data[data['Label_binary']==1]

In [7]:
Attack_data.shape

(557646, 21)

In [8]:
print(data.shape)

(2830743, 21)


In [9]:
duplicates_cols = []
duplicate_cols_cnt = 0
for column in data.columns:
    if data[column].is_unique == False:
        duplicates_cols.append(column)
        duplicate_cols_cnt += 1
print(duplicates_cols)
print('\n')
print('The number of duplicate columns are:', duplicate_cols_cnt)

['Destination Port', 'Flow Duration', 'Fwd Packet Length Max', 'Bwd Packet Length Std', 'Flow IAT Std', 'Flow IAT Min', 'Fwd IAT Mean', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Std', 'Fwd Packets/s', 'Packet Length Mean', 'Packet Length Std', 'PSH Flag Count', 'Average Packet Size', 'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Active Std', 'Label_binary']


The number of duplicate columns are: 21


In [34]:
dest_port_max = np.argmax(data['Destination Port'])

In [35]:
print(dest_port_max)

755362


In [15]:
dest_port  = data[data['Destination Port'].duplicated()]

In [16]:
dest_port['Destination Port'].unique(), dest_port['Destination Port'].nunique()

(array([0.00187686, 0.00675975, 0.00122072, ..., 0.073991  , 0.12082094,
        0.69179828]),
 42617)

In [91]:
def getIndex(data, value):
    
    elemPos = []
    result = data.isin([value])
    #series that contain the column names and the boolean True/False depending on if it is in the dataframe column
    series_boolean = result.any()
    #list that contains the name of the columns in which the value is in 
    columnNames = list(series_boolean[series_boolean == True].index)
    print('The value ', value,' is present in ', len(columnNames),' column/s')
    
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            elemPos.append((row,col))
            
    return elemPos
  

In [84]:
val = getIndex(data, value)
print('The number of occurences is: ', len(val))
print('Index positions of ', value, 'in data is: ')
for i in range(len(val)):
    print(val[i])

The value is present in  1  column/s
The number of occurences is:  16941
Index positions of  0.0003356984817273 in data is: 
(0, 'Destination Port')
(320, 'Destination Port')
(447, 'Destination Port')
(448, 'Destination Port')
(465, 'Destination Port')
(1131, 'Destination Port')
(1132, 'Destination Port')
(1133, 'Destination Port')
(1135, 'Destination Port')
(1465, 'Destination Port')
(1466, 'Destination Port')
(1467, 'Destination Port')
(1470, 'Destination Port')
(1478, 'Destination Port')
(1479, 'Destination Port')
(1480, 'Destination Port')
(1482, 'Destination Port')
(7312, 'Destination Port')
(7321, 'Destination Port')
(7322, 'Destination Port')
(7327, 'Destination Port')
(7328, 'Destination Port')
(8647, 'Destination Port')
(8650, 'Destination Port')
(8651, 'Destination Port')
(8653, 'Destination Port')
(9949, 'Destination Port')
(9951, 'Destination Port')
(9953, 'Destination Port')
(9955, 'Destination Port')
(14077, 'Destination Port')
(14095, 'Destination Port')
(14096, 'Destina

(462928, 'Destination Port')
(462941, 'Destination Port')
(462942, 'Destination Port')
(463405, 'Destination Port')
(463406, 'Destination Port')
(463413, 'Destination Port')
(463414, 'Destination Port')
(463758, 'Destination Port')
(463763, 'Destination Port')
(463779, 'Destination Port')
(463780, 'Destination Port')
(463795, 'Destination Port')
(463796, 'Destination Port')
(463811, 'Destination Port')
(463812, 'Destination Port')
(464742, 'Destination Port')
(464747, 'Destination Port')
(464750, 'Destination Port')
(464755, 'Destination Port')
(465198, 'Destination Port')
(465203, 'Destination Port')
(465230, 'Destination Port')
(465235, 'Destination Port')
(465325, 'Destination Port')
(465326, 'Destination Port')
(465341, 'Destination Port')
(465344, 'Destination Port')
(465553, 'Destination Port')
(465556, 'Destination Port')
(465557, 'Destination Port')
(465558, 'Destination Port')
(466683, 'Destination Port')
(466684, 'Destination Port')
(466693, 'Destination Port')
(466696, 'Dest

(998504, 'Destination Port')
(998538, 'Destination Port')
(998539, 'Destination Port')
(998540, 'Destination Port')
(998542, 'Destination Port')
(998548, 'Destination Port')
(998550, 'Destination Port')
(998551, 'Destination Port')
(998553, 'Destination Port')
(998900, 'Destination Port')
(998902, 'Destination Port')
(998903, 'Destination Port')
(998909, 'Destination Port')
(999052, 'Destination Port')
(999055, 'Destination Port')
(999056, 'Destination Port')
(999059, 'Destination Port')
(999139, 'Destination Port')
(999141, 'Destination Port')
(999142, 'Destination Port')
(999147, 'Destination Port')
(999160, 'Destination Port')
(999161, 'Destination Port')
(999162, 'Destination Port')
(999164, 'Destination Port')
(1001507, 'Destination Port')
(1001511, 'Destination Port')
(1001512, 'Destination Port')
(1001519, 'Destination Port')
(1001532, 'Destination Port')
(1001535, 'Destination Port')
(1001540, 'Destination Port')
(1001542, 'Destination Port')
(1003285, 'Destination Port')
(1003

(1505673, 'Destination Port')
(1505674, 'Destination Port')
(1505888, 'Destination Port')
(1505889, 'Destination Port')
(1505890, 'Destination Port')
(1505891, 'Destination Port')
(1512332, 'Destination Port')
(1512397, 'Destination Port')
(1512398, 'Destination Port')
(1512399, 'Destination Port')
(1512400, 'Destination Port')
(1512533, 'Destination Port')
(1512534, 'Destination Port')
(1512535, 'Destination Port')
(1513300, 'Destination Port')
(1513301, 'Destination Port')
(1513302, 'Destination Port')
(1513303, 'Destination Port')
(1520714, 'Destination Port')
(1520715, 'Destination Port')
(1520716, 'Destination Port')
(1520717, 'Destination Port')
(1520772, 'Destination Port')
(1520773, 'Destination Port')
(1520774, 'Destination Port')
(1520775, 'Destination Port')
(1521202, 'Destination Port')
(1521203, 'Destination Port')
(1521204, 'Destination Port')
(1521205, 'Destination Port')
(1521539, 'Destination Port')
(1521540, 'Destination Port')
(1521541, 'Destination Port')
(1521542, 

(1694170, 'Destination Port')
(1694179, 'Destination Port')
(1694200, 'Destination Port')
(1694201, 'Destination Port')
(1694204, 'Destination Port')
(1694206, 'Destination Port')
(1694207, 'Destination Port')
(1694290, 'Destination Port')
(1694478, 'Destination Port')
(1694488, 'Destination Port')
(1694494, 'Destination Port')
(1694613, 'Destination Port')
(1694649, 'Destination Port')
(1694664, 'Destination Port')
(1694665, 'Destination Port')
(1694667, 'Destination Port')
(1694669, 'Destination Port')
(1694736, 'Destination Port')
(1694761, 'Destination Port')
(1694764, 'Destination Port')
(1694766, 'Destination Port')
(1694767, 'Destination Port')
(1694775, 'Destination Port')
(1694779, 'Destination Port')
(1694780, 'Destination Port')
(1694781, 'Destination Port')
(1694784, 'Destination Port')
(1694786, 'Destination Port')
(1694787, 'Destination Port')
(1694799, 'Destination Port')
(1694857, 'Destination Port')
(1694868, 'Destination Port')
(1694870, 'Destination Port')
(1694880, 

(1723825, 'Destination Port')
(1723846, 'Destination Port')
(1723864, 'Destination Port')
(1723865, 'Destination Port')
(1723912, 'Destination Port')
(1723929, 'Destination Port')
(1723953, 'Destination Port')
(1723966, 'Destination Port')
(1723967, 'Destination Port')
(1723974, 'Destination Port')
(1723983, 'Destination Port')
(1724000, 'Destination Port')
(1724006, 'Destination Port')
(1724007, 'Destination Port')
(1724013, 'Destination Port')
(1724036, 'Destination Port')
(1724043, 'Destination Port')
(1724142, 'Destination Port')
(1724160, 'Destination Port')
(1724167, 'Destination Port')
(1724170, 'Destination Port')
(1724176, 'Destination Port')
(1724179, 'Destination Port')
(1724196, 'Destination Port')
(1724341, 'Destination Port')
(1724410, 'Destination Port')
(1724411, 'Destination Port')
(1724493, 'Destination Port')
(1724518, 'Destination Port')
(1724519, 'Destination Port')
(1724530, 'Destination Port')
(1724534, 'Destination Port')
(1724539, 'Destination Port')
(1724569, 

(1849146, 'Destination Port')
(1849163, 'Destination Port')
(1849257, 'Destination Port')
(1849277, 'Destination Port')
(1849424, 'Destination Port')
(1849533, 'Destination Port')
(1849592, 'Destination Port')
(1849721, 'Destination Port')
(1849725, 'Destination Port')
(1849794, 'Destination Port')
(1849799, 'Destination Port')
(1849858, 'Destination Port')
(1850015, 'Destination Port')
(1850050, 'Destination Port')
(1850065, 'Destination Port')
(1850080, 'Destination Port')
(1850156, 'Destination Port')
(1850326, 'Destination Port')
(1850418, 'Destination Port')
(1850431, 'Destination Port')
(1850450, 'Destination Port')
(1850456, 'Destination Port')
(1850651, 'Destination Port')
(1850706, 'Destination Port')
(1850708, 'Destination Port')
(1850738, 'Destination Port')
(1850801, 'Destination Port')
(1850902, 'Destination Port')
(1850907, 'Destination Port')
(1851000, 'Destination Port')
(1851051, 'Destination Port')
(1851131, 'Destination Port')
(1851150, 'Destination Port')
(1851161, 

(1964167, 'Destination Port')
(1964206, 'Destination Port')
(1964218, 'Destination Port')
(1964248, 'Destination Port')
(1964398, 'Destination Port')
(1964405, 'Destination Port')
(1964421, 'Destination Port')
(1964488, 'Destination Port')
(1964501, 'Destination Port')
(1964521, 'Destination Port')
(1964727, 'Destination Port')
(1964797, 'Destination Port')
(1964818, 'Destination Port')
(1964828, 'Destination Port')
(1964847, 'Destination Port')
(1964877, 'Destination Port')
(1964934, 'Destination Port')
(1964951, 'Destination Port')
(1964977, 'Destination Port')
(1965067, 'Destination Port')
(1965094, 'Destination Port')
(1965156, 'Destination Port')
(1965179, 'Destination Port')
(1965180, 'Destination Port')
(1965229, 'Destination Port')
(1965323, 'Destination Port')
(1965370, 'Destination Port')
(1965384, 'Destination Port')
(1965412, 'Destination Port')
(1965416, 'Destination Port')
(1965447, 'Destination Port')
(1965566, 'Destination Port')
(1965654, 'Destination Port')
(1965665, 

(2453471, 'Destination Port')
(2453472, 'Destination Port')
(2454673, 'Destination Port')
(2454674, 'Destination Port')
(2454675, 'Destination Port')
(2454723, 'Destination Port')
(2454724, 'Destination Port')
(2454836, 'Destination Port')
(2454837, 'Destination Port')
(2454854, 'Destination Port')
(2454855, 'Destination Port')
(2455651, 'Destination Port')
(2455652, 'Destination Port')
(2455655, 'Destination Port')
(2455656, 'Destination Port')
(2455957, 'Destination Port')
(2455958, 'Destination Port')
(2455966, 'Destination Port')
(2455967, 'Destination Port')
(2455972, 'Destination Port')
(2455973, 'Destination Port')
(2455978, 'Destination Port')
(2455979, 'Destination Port')
(2456225, 'Destination Port')
(2456238, 'Destination Port')
(2456239, 'Destination Port')
(2456240, 'Destination Port')
(2456614, 'Destination Port')
(2456615, 'Destination Port')
(2456629, 'Destination Port')
(2456630, 'Destination Port')
(2456843, 'Destination Port')
(2456844, 'Destination Port')
(2456845, 

In [85]:
data.head()

,Destination Port,Flow Duration,Fwd Packet Length Max,Bwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Mean,Fwd IAT Min,Bwd IAT Mean,Bwd IAT Std,Fwd Packets/s,Packet Length Mean,Packet Length Std,PSH Flag Count,Average Packet Size,Avg Bwd Segment Size,Subflow Fwd Packets,Init_Win_bytes_forward,Init_Win_bytes_backward,Active Std,Label_binary,State
0,0.000336,1.491667e-06,0.000000,0.0,0.000000e+00,1.500000e-06,0.000000,9.999999e-08,0.000000e+00,0.0,2.008032e-03,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.004440,0.003723,0.000000,0,1
1,0.917800,7.999999e-07,0.000000,0.0,1.250775e-07,4.000000e-07,0.000000,9.999999e-08,4.083333e-07,0.0,4.016064e-03,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.003723,0.004440,0.000000,0,1
2,0.001877,8.329999e-04,0.001934,0.0,0.000000e+00,8.330082e-04,0.000000,9.999999e-08,0.000000e+00,0.0,3.335101e-06,0.014384,0.0,0.0,0.018493,0.008275,0.000000,0.000000,0.000000,0.000000,0,1
3,0.001877,3.085833e-04,0.001934,0.0,0.000000e+00,3.085916e-04,0.000000,9.999999e-08,0.000000e+00,0.0,9.004872e-06,0.014384,0.0,0.0,0.018493,0.008275,0.000000,0.000000,0.000000,0.000000,0,1
4,0.000000,9.263445e-01,0.000000,0.0,2.995055e-02,1.166667e-07,0.006345,9.999999e-08,0.000000e+00,0.0,4.408007e-07,0.000000,0.0,0.0,0.000000,0.000000,0.000664,0.000000,0.000000,0.028615,0,1


In [87]:
data['State'].nunique()

1

# DQN class

In [1]:
class DQN(nn.Module):
    def __init__(self, states, num_actions):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features = states , out_features = 10  )
        self.fc2 = nn.Linear(in_features = 10 , out_features = 15 )
        self.fc3 = nn.Linear(in_features = 15 , out_features = 2 )
        self.out = nn.Linear(in_features = 2 ,out_features = num_actions )
        
    def forward(self, t):
        t = t.flatten(start_dim =1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = F.relu(self.fc3(t))
        t = self.out(t)
        return t
    

NameError: name 'nn' is not defined

# Experience class

In [ ]:
Experience = namedtuple(
        'Experience',
            ('state', 'action', 'next_state', 'reward')
)

# Replay memory class

In [ ]:
class ReplayMemory():
    def __init__(self,capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0  
        
    def push(self, experience):
        if len(self.memory) < len(self.capacity):
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] =experience
        self.push_count += 1
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

# Epsilon Greedy strategy

In [ ]:
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
    
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * math.exp(-1. * current_step * self.decay)

# Reinforcement learning agent

In [ ]:
class Agent():
    def __init__(self, strategy, num_actions):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        
    def select_action(self, state, policy_net):
        rate = strategy.get_exploration_rate(self.current_step)
        self.current_step += 1
        
        if rate > random.random():
            return random.randrange(self.num_actions) #explore
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).item() #exploit

# Extract tensors

In [ ]:
def extract_tensors(experiences):
    batch = Experience(*zip(*experiences))
    
    t1= torch.cat(batch.state)
    t2= torch.cat(batch.action)
    t3= torch.cat(batch.reward)
    t4= torch.cat(batch.next_state)
    
    return (t1,t2,t3,t4)

# States assignment class

In [ ]:
class state_assignment():
    
    def shuffle(data):
        data=data.sample(frac=1).reset_index(drop=True)
    return data

    

# Environment class

In [ ]:
class dataset_environment():
    def __init__(self):
        self.action_space = [0,1]
        self.num_actions = len(self.action_space)
        self.states = self.define_all_states(data)
        self.num_states = len(self.states)
        self.data = data
        self.index = self.get_index
        self.last_data_index = len(self.data) - 1
        
    #function that takes the columns of dataframe and turns them into arrays stored in a single array
    def dataframe_to_arr(data): 
        data_arr = []
        row = 0
        for col in data.columns:
            row += 1
            col_arr = data[col].to_numpy()
            data_arr.append(col_arr)
    return data_arr
    
    
    #function that defines the states of a dataset and break the data into seperate states
    def define_all_states(self, data):
        
    
    #function that gets the index of the datasets
    def get_index(self, data):
    
    
    def get_next_state_reward(self,action,):
        
        done = True if self.index == self.last_data_index-1 else False
    

# Calculating Q_values

In [ ]:
class QValues():
    
    @staticmethod
    def get_current(policy_net, states, actions):
        return policy_net(states).gather(dim=1, index=actions.unsqueeze(-1))
    
    @staticmethod
    def get_next(target_net, next_states):
        

# Main program

In [ ]:
#Tune and experiment with different values
batch_size = 256
gamma = 0.999 #discount factor used in bellman's equation
eps_start = 1 #Starting value of the exploration rate
eps_end = 0.01
eps_decay = 0.001
target_update = 10  #how frequently in terms of episodes we will update the target_network's weights with the policy network weights
memory_size = 100000
lr = 0.0001 #learning rate
num_episodes = 10000 


In [ ]:
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, num_actions)
memory = ReplayMemory(memory_size)

We need to create two objects of the DQN class. a policy_net object and a target_net object. The target_net object clones the DQN model and is used to predict the Qvalues based on the next-states that can be passed to the mode. This results in the next Qvalue which is used in the Bellmann calculation.

In [ ]:
policy_net = DQN( states, num_actions)
target_net  = DQN(states, num_actions)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval() #This network is not in training mode
optimizer = optim.Adam(params = policy_net.parameters(), lr = lr)

Now we create 2 for loops, one for the episodes and the other for the timesteps. We have to reset the environment, take an action, update the tuples experience, update the reward and next states

In [ ]:
episode_durations = []
for episode in range(num_episodes):
    #env.reset() reset the environment
    state = env.get_state()
    
    for timestep in count():
        action = agent.select_action(state, policy_net)
        reward = env.take_action(action)
        next_state = env.get_state()
        memory.push(Experience(state, action, next_state, reward))
        state = next_state
        
        
        if memory.can_provide_sample(batch_size):
            experiences = memory.sample(batch_size)
            states, actions, rewards, next_states = extract_tensors(experiences)
            
            current_q_values = QValues.get_current(policy_net, states, actions)
            next_q_values = QValues.get_next(target_net, next_states)
            target_q_values = (next_q_values * gamma) + rewards
            
            loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if episode % target_update == 0:
            target_net.load_state_dict(policy_net.state_dict())